# Harmonics Demo
### See spectrum or spectrogram 
### Modify frequency and amplitude of a sine - square - sawtooth signal

Author: Dirk Van Compernolle
Created: 29/03/2021

---------------------------------------------------------

Dependencies:
- ipywidget
- matplotlib, numpy, scipy
- librosa

In [17]:
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, interactive_output
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from IPython.display import display, clear_output, Audio, HTML
import time
import librosa

def my_sine(x, w, amp, phi):
    return amp*np.sin(w * (x-phi))
#
def synth(sigtype='sin', freq=200.0, amp=1.0, Fs=8000, Tmax=0.25):
    t = np.linspace(0.0, Tmax, int(Tmax*Fs), endpoint=False)
    if sigtype == 'sin':
        x = np.sin(2.0*np.pi*freq*t)
    elif sigtype == 'square':
        x = signal.square(2.0*np.pi*freq*t)
    elif sigtype == 'sawtooth':
        x = signal.sawtooth(2.0*np.pi*freq*t)
    else:
        print( 'signal: Unrecognized signal type')
    return amp*x, t

def spectrogram(x,samplerate=8000,length=30.0,shift=10.0):
    hop_length = int(samplerate*shift/1000.)
    win_length = int(samplerate*length/1000.)
    spg_stft = librosa.stft(x,n_fft=512,hop_length=hop_length,win_length=win_length)
    return( librosa.power_to_db(np.abs(spg_stft)**2) )

# a default boxed layout
def box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

# The following code will increase the default width of your Jupyter notebook cells
# Supposed to work well 
display(HTML(data="""
<style>
    div#notebook-container    { width: 99%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

## DEMO 1: Waveform and Spectrum of Harmonic Signals

In [33]:
class Harmonics_Demo(widgets.HBox):

    def __init__(self):
        super().__init__()
        
        self.amp = 1.
        self.disptype = 'spectrum'
        self.sigtype = 'sin'
        self.freq = 440.
        self.samplerate = 16000
        self.Tmax = .25
        
        display_types = ['spectrum','spectrogram']
        signal_types = [ 'sin', 'square', 'sawtooth' ]
        self.wg_disptype = widgets.Dropdown(options=display_types,value=self.disptype,description="Display")
        self.wg_sigtype = widgets.Dropdown(options=signal_types,value=self.sigtype,description="Signal")
        self.wg_amp = widgets.FloatSlider(value=self.amp,step=0.05,max=1.0,description='Amplitude',continous_update=False)
        self.wg_freq = widgets.FloatSlider(value=self.freq,step=10.,min=100.0,max=1000.,description='Frequency',continous_update=False)
        self.wg_disptype.observe(self.disptype_observe,'value')
        self.wg_sigtype.observe(self.sigtype_observe,'value')
        self.wg_amp.observe(self.amp_observe,'value')    
        self.wg_freq.observe(self.freq_observe,'value')    
        def on_button_clicked(b):
            with self.logit:     
                clear_output()   
        self.btn_clear_log = widgets.Button(description='Clear the log')
        self.btn_clear_log.on_click(on_button_clicked)
        self.UI = widgets.VBox( [self.wg_disptype,self.wg_sigtype, self.wg_amp, self.wg_freq, self.btn_clear_log] ,layout=box_layout())
        
        self.screen = widgets.Output()
        self.screen.layout = box_layout()
        self.audio = widgets.Output()
        self.logit = widgets.Output()
        
        self.fig,self.ax = plt.subplots(2,1,constrained_layout=True, figsize=(8, 4))

        # avoid output of dummy figure on startup
        plt.close()
        # initialize the displayed function
        self.update()
        # time.sleep(2)
        self.children = [ self.screen, widgets.VBox([self.UI, self.audio,  self.logit], layout=box_layout()) ] 

    def update(self):
        y,x = synth(sigtype=self.sigtype,freq=self.freq,amp=self.amp,Tmax=self.Tmax)

        ax = self.ax
        ax[0].cla()
        ax[0].plot(x, y, color='C0')
        ax[0].set_ylim([-1, 1])
        ax[0].grid(True)
        ax[0].set_title('Waveform')
        ax[0].set_xlabel('Time(sec)')
        ax[0].set_xlim([0,self.Tmax])
       
        if self.disptype == 'spectrum':
            ax[1].cla()
            freq_ax,spec = signal.periodogram(y,fs=self.samplerate,scaling='spectrum')
            ax[1].plot(freq_ax,np.sqrt(2*spec))
            ax[1].set_title('Spectrum')
            ax[1].set_xlabel('Frequency')
            ax[1].set_ylim([0, 1])
            ax[1].grid(True)
            ax[1].set_xlim([0,self.samplerate/2.])
        elif self.disptype == 'spectrogram':
            ax[1].cla()
            spg = spectrogram(y,samplerate=self.samplerate)
            ax[1].imshow(spg[:,:-1],cmap='jet',aspect='auto',origin='lower')
            ax[1].set_title('Spectrogram')
            ax[1].set_xlabel('Time(sec)')

            
        # here come the things that go to dedicated output widgets
        with self.screen:
            clear_output(wait=True)
            display(self.fig)
        with self.audio:
            clear_output(wait=True)
            try:
                display(Audio(data=y,rate=self.samplerate,normalize=False, autoplay=True))
            except: 
                try:
                    with self.logit:
                        #clear_output(wait=True)
                        print("Warning: playing normalized sound")
                    display(Audio(data=y,rate=self.samplerate, autoplay=True))
                except: pass
     
    def disptype_observe(self,change):
        self.disptype = change.new
        self.update()
    
    def sigtype_observe(self,change):
        self.sigtype = change.new
        self.update()
    
    def amp_observe(self,change):
        self.amp = change.new
        self.update()
        
    def freq_observe(self,change):
        self.freq = change.new
        self.update()
        
Harmonics_Demo() 

Harmonics_Demo(children=(Output(layout=Layout(border='solid 1px black', margin='0px 10px 10px 0px', padding='5…